In [165]:
import numpy as np
import scipy as sp
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import numpy.random as rnd
rnd.seed(42)

from itertools import accumulate, chain

# Utils

In [296]:
def random_walk(size, mean=0, sd=1, start=0):
    sample = np.cumsum(rnd.normal(loc=mean, scale=sd, size=size))
    return pd.Series(sample + start - sample[0])
    
def slopes(limit, apfreq, mean=0, sd=1):
    counts = np.ceil(rnd.exponential(size=limit, scale=apfreq)).astype(np.int64)
    counts = counts[counts.cumsum() <= limit]
    counts = np.append(counts, limit - counts.sum())
    slopes = rnd.normal(size=len(counts), loc=mean, scale=sd)
    return pd.DataFrame({'counts': counts, 'slopes': slopes})
    
def piecewise_linear(size, slopes, start=0):
    return slopes.apply(lambda x: linear(int(x[0]), a=x[1]), axis=1, reduce=True)
    
def linear(size, a=1, b=0):
    return a * (np.ones(size).cumsum() - 1) + b

In [294]:
linear(100, 1)

0      0.0
1      1.0
2      2.0
3      3.0
4      4.0
      ... 
95    95.0
96    96.0
97    97.0
98    98.0
99    99.0
Length: 100, dtype: float64

In [289]:
sl = slopes(100, 20)

In [297]:
sl.apply(lambda x: linear(int(x[0]), a=x[1]), axis=1, reduce=True)

/home/textor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The reduce argument is deprecated and will be removed in a future version. You can specify result_type='reduce' to try to reduce the result to the original dimensions
  """Entry point for launching an IPython kernel.


0    [0.0, -1.8287275989469451, -3.6574551978938903...
1              [0.0, 0.4914357661724, 0.9828715323448]
2    [0.0, 0.8617069780326444, 1.7234139560652888, ...
3        [0.0, -1.700752193853906, -3.401504387707812]
dtype: object